### 와인 전체 특성을 사용한 훈련, 예측, 평가하기

In [17]:
### 와인 전체 데이터셋 읽어들이기
from sklearn.datasets import load_wine

wine = load_wine()

In [2]:
"""
<딕셔너리에서 사용하는 key>
 - feature_name : 특성이름
 - data : 특성에 대한 데이터(특성 데이터, 독립변수)
 - target : 종속변수
 - target_names : 종속변수 범주 데이터
                : class_0 : 레드와인
                : class_1 : 화이트와인
                : class_2 : ? (사용안함)

<와인 성분에 포함된 화학조성 특성>
'alcohol' : 알콜도수
'malic_acid' : 말산
'ash' : 회분
'alcalinity_of_ash' : 회분의 알칼리도
'magnesium' : 마그네슘
'total_phenols' : 총 포리페놀
'flavanoids' : 플라보노이드 폴리페놀
'nonflavanoid_phenols' : 비 프라보노이드 폴리페놀
'proanthocyanins' : 프로안토시아닌
'color_intensity' : 색상 강도
'hue' : 색상
'od280/od315_of_diluted_wines' : 희석와인의 비율
'proline : 프롤린
"""

"\n<딕셔너리에서 사용하는 key>\n - feature_name : 특성이름\n - data : 특성에 대한 데이터(특성 데이터, 독립변수)\n - target : 종속변수\n - target_names : 종속변수 범주 데이터\n                : class_0 : 레드와인\n                : class_1 : 화이트와인\n                : class_2 : ? (사용안함)\n\n<와인 성분에 포함된 화학조성 특성>\n'alcohol' : 알콜도수\n'malic_acid' : 말산\n'ash' : 회분\n'alcalinity_of_ash' : 회분의 알칼리도\n'magnesium' : 마그네슘\n'total_phenols' : 총 포리페놀\n'flavanoids' : 플라보노이드 폴리페놀\n'nonflavanoid_phenols' : 비 프라보노이드 폴리페놀\n'proanthocyanins' : 프로안토시아닌\n'color_intensity' : 색상 강도\n'hue' : 색상\n'od280/od315_of_diluted_wines' : 희석와인의 비율\n'proline : 프롤린\n"

In [10]:
 ### 튜닝 자리 
gridParams={}
gridParams["max_depth"] = [50, 100, 150, 200]
gridParams["C"] = 0.5
gridParams["random_state"] = [42]
n_jobs = -1

In [ ]:


import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 모델 리스트
model_list = [
    KNeighborsClassifier(),
    LogisticRegression(max_iter=500),
    DecisionTreeClassifier()
]

# 1. 데이터 로드
wine = load_wine()
X, y = wine.data, wine.target

# 2. class_0과 class_1만 필터링
hi = (y == 0) | (y == 1)
X_fil = X[hi]
y_fil = y[hi]

# 3. 데이터 분할 (8:2)
train_input, test_input, train_target, test_target = train_test_split(
    X_fil, y_fil, test_size=0.2, random_state=42, stratify=y_fil
)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

# 특성공학 (PolynomialFeatures 적용)
for degree in range(1, 5):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    train_poly = poly.fit_transform(train_input)
    test_poly = poly.transform(test_input)

    # 스케일링 적용
    scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
    scaler_names = ["StandardScaler", "MinMaxScaler", "RobustScaler"]

    for scaler, scaler_name in zip(scalers, scaler_names):
        train_scaled = scaler.fit_transform(train_poly)
        test_scaled = scaler.transform(test_poly)

        for model in model_list:
            model_name = model.__class__.__name__  # 모델 이름 확인

            # 모델별 하이퍼파라미터 설정
            param_grid = {}

            if model_name == "KNeighborsClassifier":
                param_grid = {"n_neighbors": [3, 5, 7]}

            elif model_name == "LogisticRegression":
                param_grid = {"C": [0.01, 0.1, 1, 10], "max_iter": [500]}

            elif model_name == "DecisionTreeClassifier":
                param_grid = {"max_depth": [3, 5, 10]}

            # GridSearchCV 실행
            grid_model = GridSearchCV(
                estimator=model,
                param_grid=param_grid,
                cv=5,
                n_jobs=-1
            )

            grid_model.fit(train_scaled, train_target)

            # 모델 성능 평가
            train_score = grid_model.score(train_scaled, train_target)
            test_score = grid_model.score(test_scaled, test_target)

            print(f"모델: {model_name} | 차원: {degree} | 스케일링: {scaler_name}")
            print(f"최적 하이퍼파라미터: {grid_model.best_params_}")
            print(f"훈련 정확도: {train_score:.4f}, 테스트 정확도: {test_score:.4f}")
            print(f"과적합 여부: {train_score - test_score}")
            print()
            
            
### (해석) 가장 좋은 모델 
# 모델: KNeighborsClassifier | 차원: 3 | 스케일링: MinMaxScaler
# 최적 하이퍼파라미터: {'n_neighbors': 5}
# 훈련 정확도: 0.9808, 테스트 정확도: 0.9615
# 과적합 여부: 0.019230769230769162            

(104, 13) (104,)
(26, 13) (26,)
모델: KNeighborsClassifier | 차원: 1 | 스케일링: StandardScaler
최적 하이퍼파라미터: {'n_neighbors': 5}
훈련 정확도: 0.9808, 테스트 정확도: 0.9615
과적합 여부: 0.019230769230769162

모델: LogisticRegression | 차원: 1 | 스케일링: StandardScaler
최적 하이퍼파라미터: {'C': 0.1, 'max_iter': 500}
훈련 정확도: 1.0000, 테스트 정확도: 0.9615
과적합 여부: 0.038461538461538436

모델: DecisionTreeClassifier | 차원: 1 | 스케일링: StandardScaler
최적 하이퍼파라미터: {'max_depth': 10}
훈련 정확도: 1.0000, 테스트 정확도: 0.9231
과적합 여부: 0.07692307692307687

모델: KNeighborsClassifier | 차원: 1 | 스케일링: MinMaxScaler
최적 하이퍼파라미터: {'n_neighbors': 7}
훈련 정확도: 0.9904, 테스트 정확도: 0.9615
과적합 여부: 0.028846153846153855

모델: LogisticRegression | 차원: 1 | 스케일링: MinMaxScaler
최적 하이퍼파라미터: {'C': 1, 'max_iter': 500}
훈련 정확도: 0.9904, 테스트 정확도: 0.9615
과적합 여부: 0.028846153846153855

모델: DecisionTreeClassifier | 차원: 1 | 스케일링: MinMaxScaler
최적 하이퍼파라미터: {'max_depth': 5}
훈련 정확도: 1.0000, 테스트 정확도: 0.9231
과적합 여부: 0.07692307692307687

모델: KNeighborsClassifier | 차원: 1 | 스케일링: RobustScaler
최적 하이퍼파라미터: {'n